In [ ]:

import openai, json, logging
from openai import OpenAI
import os
import random
from random import sample
import json
from tqdm.notebook import tqdm
import evaluate
seqeval = evaluate.load('seqeval')

In [ ]:
#Import the data
#Import the positive examples
green = []
with open('./Data/Supervised/Green/test.json', 'r') as f:
    for line in f:
        green.append(json.loads(line.strip()))
say = []
with open('./Data/Supervised/Sayfullina/test.json', 'r') as f:
    for line in f:
        say.append(json.loads(line.strip()))
ss = []
with open('./Data/Supervised/SkillSpan/test.json', 'r') as f:
    for line in f:
        ss.append(json.loads(line.strip()))

In [5]:
#samples for GPT-4
random.seed(2137)
green_300= random.sample(green, 300)
ss_300= random.sample(ss, 300)
say_300= random.sample(say, 300)

In [ ]:
client = OpenAI(
    # Insert your OpenAI API key here
    api_key=(""),
)

In [ ]:
instruction_system = (
    "You are an information-extraction assistant.\n"
    "Task: given ONE tokenized job-advert sentence, output a BIO tag for "
    "every token describing a skill or a competence. A tag must be exactly \"B\", \"I\", or \"O\".\n"
    "Return ONLY a JSON object of the form: "
    "\'{\"predicted_spans\": [\"B\", \"I\", \"O\", ...]}\'.\n"
    "Do NOT add any other keys, comments, or text.\n"
    "Make sure the length of the \"predicted_spans\" matches the length of tokens.\n"
    "If the user supplies example tokens-tags pairs first, follow the provided pattern."
)

instruction_system_say = (
    "You are an information-extraction assistant.\n"
    "Task: given ONE tokenized job-advert sentence, output a BIO tag for "
    "every token describing a SOFT skill or a competence. A tag must be exactly \"B\", \"I\", or \"O\".\n"
    "Return ONLY a JSON object of the form: "
    "\'{\"predicted_spans\": [\"B\", \"I\", \"O\", ...]}\'.\n"
    "Do NOT add any other keys, comments, or text.\n"
    "Make sure the length of the \"predicted_spans\" matches the length of tokens.\n"
    "If the user supplies example tokens-tags pairs first, follow the provided pattern."
)


TOOLS = [{
    "type": "function",
    "function": {
        "name": "tagger",
        "description": "Return BIO tags for a token list",
        "parameters": {
            "type": "object",
            "properties": {
                "predicted_spans": {
                    "type": "array",
                    "items": {"type": "string", "enum": ["B", "I", "O"]},
                    "description": "BIO tag per token"
                }
            },
            "required": ["predicted_spans"]
        }
    }
}]

FEW_SHOT_EXAMPLES_SS = [
    {"tokens": ["Webpack", "React", "&", "Redux"], "predicted_spans": ["B", "B", "O", "B"]},
    {"tokens": ["You", "will", "have", "experience", "with", "application", "design", "&", "architecture", "using", "modern", "design", "patterns"], "predicted_spans": ["O", "O", "O", "O", "O", "B", "I", "I", "I", "O", "B", "I", "I"]},
    {"tokens": ["Become", "a", "part", "of", "our", "team"], "predicted_spans": ["O", "O", "O", "O", "O", "O"]},
    {"tokens": ["About", "<ORGANIZATION>", "<ORGANIZATION>", "<ORGANIZATION>", "<ORGANIZATION>", "is", "the", "hub", "for", "innovation", "at", "<ORGANIZATION>", "<ORGANIZATION>", "<ORGANIZATION>", "<ORGANIZATION>", "<ORGANIZATION>", "(", "<ORGANIZATION>", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]},
    {"tokens": ["<ORGANIZATION>", "is", "looking", "for", "a", "Senior", "Java", "Developer", "to", "build", "new", "features", "design", "and", "implement", "useful", "API", "methods", "and", "increase", "the", "reliability", "of", "our", "software", "continuously", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "B", "I", "I", "B", "I", "I", "I", "I", "I", "O", "B", "I", "I", "I", "I", "I", "I", "O"]}
]

FEW_SHOT_EXAMPLES_GREEN = [
    {"tokens": ["This", "UK", "based", "staff", "position", "is", "within", "the", "technical", "directorate", "team", "with", "the", "successful", "candidate", "gaining", "the", "responsibility", "for", "the", "operations", "of", "performance", "optimisation", "teams", "and", "shall", "have", "the", "opportunity", "to", "review", "and", "implement", "improvements", "to", "operational", "practise", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "O", "B", "I", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B", "I", "O", "O", "O", "O", "O", "O", "O", "B", "I", "I", "I", "I", "I", "I", "O"]},
    {"tokens": ["Rare", "opening", "for", "a", "highcalibre", "assistant", "at", "this", "prestigious", "City", "firm", "with", "a", "highly", "regarded", "commercial", "property", "department", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]},
    {"tokens": ["topologies", "as", "well", "as", "surface", "mount", "assembly", "is", "essential", "."], "predicted_spans": ["B", "I", "I", "I", "I", "I", "I", "O", "O", "O"]},
    {"tokens": ["Responsibilities", "will", "be", "an", "interesting", "split", "of", "3rd", "/", "4th", "line", "support", "to", "project", "work", "\"", "giving", "the", "incumbent", "some", "fantastic", "exposure", "to", "design", "and", "planning", "of", "infrastructure", "projects", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "B", "I", "I", "I", "I", "O", "B", "I", "O", "O", "O", "O", "O", "O", "O", "O", "B", "I", "I", "I", "I", "I", "O"]},
    {"tokens": ["You", "may", "also", "be", "involved", "in", "Method", "Development", "and", "Validation", "."], "predicted_spans": ["O", "O", "O", "O", "O", "O", "B", "I", "I", "I", "O"]}
]

FEW_SHOT_EXAMPLES_SAY = [
    {"tokens": ["ability", "to", "work", "under", "stress", "condition"], "predicted_spans": ["O", "O", "B", "I", "I", "O"]},
    {"tokens": ["workplace", "excellent", "interpersonal", "and", "communication", "skill", "diplomatic", "and", "negotiation", "skill", "the"], "predicted_spans": ["O", "O", "O", "O", "O", "O", "B", "O", "O", "O", "O"]},
    {"tokens": ["confidence", "in", "decision", "making", "abilities", "and", "deal", "effectively", "with", "unexpected"], "predicted_spans": ["O", "O", "B", "I", "I", "O", "O", "O", "O", "O"]},
    {"tokens": [",", "linux", "javascript", "ajax", "html", "xml", "cs", "experience", "with", "dynamic", "databasedriven", "website", "it", "relate", "degree", "this", "position", "offer", "you", "the"], "predicted_spans": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "B", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]},
    {"tokens": ["excellent", "communicator", "and", "a", "hard", "worker"], "predicted_spans": ["O", "O", "O", "O", "B", "I"]}    
]

In [162]:
def build_user_prompt(test_tokens, few_shot_examples):
    msg = []
    msg.append(f"The examples below show perfectly assigned BIO tags:\n")
    for ex in few_shot_examples:
        msg.append(f"Tokens: {ex['tokens']}\n"
                   f"Output: {{\"predicted_spans\": {ex['predicted_spans']}}}\n"
        )
    msg.append(f"Following the same pattern assign BIO tags for the following {len(test_tokens)} tokens:\n")
    msg.append(f"Tokens: {test_tokens}\nOutput: ")
    return "".join(msg)

In [163]:
model_type_3 = "gpt-3.5-turbo"
model_type_4 = "gpt-4.1"

In [ ]:
#instruction-tuned
def generate_sentences_with_word(dictionary, few_shot=False, few_shot_examples_data=None, type=None):

    output_list, failed_cases, bad = [], [], 0
    bar = tqdm(dictionary, desc="LLM tagging", unit="sent")
    
    if type == 'say':
        instruction_to_use = instruction_system_say
    else:
        instruction_to_use = instruction_system   
    for item in bar:
        tokens = item['tokens']
        gold_tags = item['tags_skill']
        
        #build user prompt 
        user_prompt = build_user_prompt(tokens, few_shot_examples_data) if few_shot else (
        f"Assign BIO tags for the following {len(tokens)} tokens:\nTokens: {tokens}"
        )
        
        try:
            response = client.chat.completions.create(
                model=model_type_3,
                messages=[
                    {"role": "system", "content": instruction_to_use},
                    {"role": "user", "content": user_prompt}
                ],
                tools = TOOLS,
                tool_choice={"type": "function", "function": {"name": "tagger"}},
                max_tokens=len(tokens) + 50,
                temperature=0.0,
                response_format={"type": "json_object"}
            )
        
            tags = response.choices[0].message.tool_calls[0].function.arguments
            predicted = json.loads(tags)["predicted_spans"]
            if len(predicted) != len(tokens):
                raise ValueError("length mismatch")
            
        except Exception as e:
            logging.warning(f"LLM error: {e}")
            predicted = ['O'] * len(tokens)
            failed_cases.append(tokens) 
            bad += 1
        
        bar.set_postfix(bad=bad)  
        output_list.append({"tokens": tokens,
                        "predicted_tags": predicted,
                        "gold_tags": gold_tags})
    bar.close()
    print(f"There were {bad} errors due to the length mismatch")
    
    return output_list, failed_cases

In [ ]:
# Green zero-shot (GPT-3.5-turbo)
green_llm, failed_g_zero = generate_sentences_with_word(green, few_shot=False)

In [ ]:
# Green zero-shot (GPT-4.1)
green_llm_4, failed = generate_sentences_with_word(green_300, few_shot=False)

In [ ]:
# Skillspan zero-shot (GPT-3.5-turbo)
ss_llm, failed_ss = generate_sentences_with_word(ss, few_shot=False)

In [ ]:
# Skillspan zero-shot (GPT-4.1)
ss_llm_4, failed_ss_4 = generate_sentences_with_word(ss_300, few_shot=False)

In [ ]:
# Sayfullina zero-shot (GPT-3.5-turbo)
say_llm_zero, failed_say_zero = generate_sentences_with_word(say, few_shot=False, type='say')

In [ ]:
# Sayfullina zero-shot (GPT-4.1)
say_llm_4, failed_say_4 = generate_sentences_with_word(say_300, few_shot=False, type='say')

## Few shot

In [ ]:
# Green few-shot (GPT-3.5-turbo)
green_llm_few, failed_g_few = generate_sentences_with_word(green, few_shot=True, few_shot_examples_data=FEW_SHOT_EXAMPLES_GREEN)

In [ ]:
# Skillspan few-shot (GPT-3.5-turbo)
ss_llm_few, failed_ss_few = generate_sentences_with_word(ss, few_shot=True, few_shot_examples_data=FEW_SHOT_EXAMPLES_SS)

In [ ]:
# Sayfullina few-shot (GPT-3.5-turbo)
say_llm_few, failed_say_few = generate_sentences_with_word(say, few_shot=True, few_shot_examples_data=FEW_SHOT_EXAMPLES_SAY, type='say')

## Evaluation

In [ ]:
def evaluate(data):
    predictions = [x['predicted_tags'] for x in data]
    true_labels = [x['gold_tags'] for x in data]
    results = seqeval.compute(predictions=predictions, references=true_labels)
    
    overall_precision = results['overall_precision'] 
    overall_recall = results['overall_recall']
    overall_f1 = results['overall_f1']
    
    print(f'F1-span score is {round(overall_f1*100, 2)}%, with {round(overall_precision*100, 2)}% precision and {round(overall_recall*100, 2)}% recall.')